In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as spst
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report,mean_absolute_percentage_error
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.impute import KNNImputer
from workalendar.asia import SouthKorea
cal = SouthKorea()
from catboost import CatBoostRegressor

In [3]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
building_info = pd.read_csv("building_info.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [3]:
data

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40
...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,20220824 19,23.1,NaN,0.9,86.0,0.5,NaN,881.04
203996,100_20220824 20,100,20220824 20,22.4,NaN,1.3,86.0,0.0,NaN,798.96
203997,100_20220824 21,100,20220824 21,21.3,NaN,1.0,92.0,NaN,NaN,825.12
203998,100_20220824 22,100,20220824 22,21.0,NaN,0.3,94.0,NaN,NaN,640.08


In [5]:
def gggg(test,building_info):
    test = test.drop(["num_date_time",'일조(hr)','일사(MJ/m2)'],axis=1)
    test["강수량(mm)"] = test["강수량(mm)"].fillna(0)
    
    imputer = KNNImputer(n_neighbors=5)
    imputer2 = KNNImputer(n_neighbors=5)    
    test[["풍속(m/s)"]] = imputer.fit_transform(test[["풍속(m/s)"]])
    test[["습도(%)"]] = imputer2.fit_transform(test[["습도(%)"]])
    
    test = pd.merge(test, building_info, how = 'inner', on = '건물번호')
    test = test.drop(['건물번호'],axis=1)
    
    test['일시1'] = pd.to_datetime(test['일시'])
    test['month'] = test["일시1"].dt.month
    test['day'] = test["일시1"].dt.day
    test['hour'] = test["일시1"].dt.hour
    test['강수 유무'] = np.where(test["강수량(mm)"]==0,0,1)
    test["불쾌지수"] = 9/5*test["기온(C)"]-0.55*(1-(test["습도(%)"]/100))*(9/5*test["기온(C)"]-26)+32
    
    test['측정일시2'] = pd.to_datetime(test['일시1']).dt.date
    test['측정일시2'] = pd.to_datetime(test['측정일시2'])
    test['day_of_week'] = test['측정일시2'].dt.day_name()  # 요일
    test = test.rename(columns={'측정일시2':'Date'})
    holiday=pd.DataFrame(cal.holidays(2022))
    holiday=holiday.rename(columns={0:"Date",1:"holiday"})
    holiday["Date"] = pd.to_datetime(holiday["Date"])
    test = pd.merge(test,holiday,on="Date",how="left")
    test['빨간날'] = np.where((test['day_of_week']=='Sunday') | (test['day_of_week']=='Saturday') | (test['holiday'].isnull()==False),1,0)
    
    test = test.drop(['holiday','day_of_week','Date','일시1','일시'],axis=1)

    test['불쾌지수 단계'] = pd.cut(test['불쾌지수'], bins =[0,68, 75, 80,100] , labels = [0,1,2,3])
    test = pd.get_dummies(test, columns = ['불쾌지수 단계'],drop_first=True)
    # test = test.drop(['연면적(m2)','냉방면적(m2)'],axis=1)
    
    test = test.drop(["태양광용량(kW)","ESS저장용량(kWh)","PCS용량(kW)"],axis=1)

    return test

def test123(test,building_info):
    test = test.drop(["num_date_time"],axis=1)
    test["강수량(mm)"] = test["강수량(mm)"].fillna(0)
    
    imputer = KNNImputer(n_neighbors=5)
    imputer2 = KNNImputer(n_neighbors=5)    
    test[["풍속(m/s)"]] = imputer.fit_transform(test[["풍속(m/s)"]])
    test[["습도(%)"]] = imputer2.fit_transform(test[["습도(%)"]])
    
    test = pd.merge(test, building_info, how = 'inner', on = '건물번호')
    test = test.drop(['건물번호'],axis=1)
    
    test['일시1'] = pd.to_datetime(test['일시'])
    test['month'] = test["일시1"].dt.month
    test['day'] = test["일시1"].dt.day
    test['hour'] = test["일시1"].dt.hour
    test['강수 유무'] = np.where(test["강수량(mm)"]==0,0,1)
    test["불쾌지수"] = 9/5*test["기온(C)"]-0.55*(1-(test["습도(%)"]/100))*(9/5*test["기온(C)"]-26)+32
    
    test['측정일시2'] = pd.to_datetime(test['일시1']).dt.date
    test['측정일시2'] = pd.to_datetime(test['측정일시2'])
    test['day_of_week'] = test['측정일시2'].dt.day_name()  # 요일
    test = test.rename(columns={'측정일시2':'Date'})
    holiday=pd.DataFrame(cal.holidays(2022))
    holiday=holiday.rename(columns={0:"Date",1:"holiday"})
    holiday["Date"] = pd.to_datetime(holiday["Date"])
    test = pd.merge(test,holiday,on="Date",how="left")
    test['빨간날'] = np.where((test['day_of_week']=='Sunday') | (test['day_of_week']=='Saturday') | (test['holiday'].isnull()==False),1,0)
    
    test = test.drop(['holiday','day_of_week','Date','일시1','일시'],axis=1)
    
    
    test['불쾌지수 단계'] = pd.cut(test['불쾌지수'], bins =[0,68, 75, 80,100] , labels = [0,1,2,3])
    test = pd.get_dummies(test, columns = ['불쾌지수 단계'],drop_first=True)
    # test = test.drop(['연면적(m2)','냉방면적(m2)'],axis=1)
    
    test = test.drop(["태양광용량(kW)","ESS저장용량(kWh)","PCS용량(kW)"],axis=1)

    return test

def result(sample_submission,y_pred):
    sample_submission['answer1'] = y_pred
    sample_submission = sample_submission.drop('answer',axis=1)
    sample_submission = sample_submission.rename(columns={'answer1':'answer'})
    sample_submission.to_csv('cat2.csv',index = False)

In [7]:
train = gggg(data,building_info)
train1 = train.loc[train['hour']==0]
train1 = pd.get_dummies(train1,columns=['건물유형'],drop_first=True)
train1

,기온(C),강수량(mm),풍속(m/s),습도(%),전력소비량(kWh),연면적(m2),냉방면적(m2),month,day,hour,...,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,18.6,0.0,0.9,42.0,1085.28,110634.00,39570.00,6,1,0,...,0,0,0,0,0,0,0,0,0,0
24,18.1,0.0,0.4,67.0,1034.40,110634.00,39570.00,6,2,0,...,0,0,0,0,0,0,0,0,0,0
48,18.9,0.0,2.1,85.0,1184.16,110634.00,39570.00,6,3,0,...,0,0,0,0,0,0,0,0,0,0
72,22.3,0.0,2.2,76.0,1203.84,110634.00,39570.00,6,4,0,...,0,0,0,0,0,0,0,0,0,0
96,24.8,0.0,2.1,60.0,963.36,110634.00,39570.00,6,5,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203880,27.5,0.0,2.3,88.0,506.16,57497.84,40035.23,8,20,0,...,0,0,0,0,0,0,0,0,0,1
203904,23.4,0.0,0.4,99.0,473.28,57497.84,40035.23,8,21,0,...,0,0,0,0,0,0,0,0,0,1
203928,24.3,0.0,0.4,97.0,493.68,57497.84,40035.23,8,22,0,...,0,0,0,0,0,0,0,0,0,1
203952,25.2,0.0,0.7,90.0,478.08,57497.84,40035.23,8,23,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
test1 = test123(test,building_info)
test1 = test1.loc[test1['hour']==0]
test1 = pd.get_dummies(test1,columns=['건물유형'],drop_first=True)
test1

,기온(C),강수량(mm),풍속(m/s),습도(%),연면적(m2),냉방면적(m2),month,day,hour,강수 유무,...,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,23.5,0.0,2.2,72.0,110634.00,39570.00,8,25,0,0,...,0,0,0,0,0,0,0,0,0,0
24,21.1,0.0,1.7,81.0,110634.00,39570.00,8,26,0,0,...,0,0,0,0,0,0,0,0,0,0
48,23.1,0.0,2.2,82.0,110634.00,39570.00,8,27,0,0,...,0,0,0,0,0,0,0,0,0,0
72,19.6,0.0,1.8,66.0,110634.00,39570.00,8,28,0,0,...,0,0,0,0,0,0,0,0,0,0
96,22.5,0.0,1.5,66.0,110634.00,39570.00,8,29,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16680,19.5,0.0,0.3,98.0,57497.84,40035.23,8,27,0,0,...,0,0,0,0,0,0,0,0,0,1
16704,15.3,0.0,0.1,96.0,57497.84,40035.23,8,28,0,0,...,0,0,0,0,0,0,0,0,0,1
16728,19.2,0.0,0.2,91.0,57497.84,40035.23,8,29,0,0,...,0,0,0,0,0,0,0,0,0,1
16752,18.4,0.0,2.2,89.0,57497.84,40035.23,8,30,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
8500*0.8

6800.0

In [10]:
list(test1)

['기온(C)',
 '강수량(mm)',
 '풍속(m/s)',
 '습도(%)',
 '연면적(m2)',
 '냉방면적(m2)',
 'month',
 'day',
 'hour',
 '강수 유무',
 '불쾌지수',
 '빨간날',
 '불쾌지수 단계_1',
 '불쾌지수 단계_2',
 '불쾌지수 단계_3',
 '건물유형_공공',
 '건물유형_대학교',
 '건물유형_데이터센터',
 '건물유형_백화점및아울렛',
 '건물유형_병원',
 '건물유형_상용',
 '건물유형_아파트',
 '건물유형_연구소',
 '건물유형_지식산업센터',
 '건물유형_할인마트',
 '건물유형_호텔및리조트']

In [11]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

x=train1.drop('전력소비량(kWh)',axis=1)
y=train1[['전력소비량(kWh)']]

x_train, x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=42)
        
firstb = x_train[["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
 '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트']]
x_ = x_train.drop(["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
             '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트'],axis=1)

firstbb = x_val[["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
 '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트']]
x__ = x_val.drop(["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
             '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트'],axis=1)



firstbbb = test1[["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
 '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트']]
x___ = test1.drop(["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
             '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트'],axis=1)

scaler = StandardScaler()
scaler.fit(x_)
x_train_ = scaler.transform(x_)
x_val_ = scaler.transform(x__) 
x_train = pd.DataFrame(x_train_,index=x_.index,columns=x_.columns)
x_val = pd.DataFrame(x_val_,index=x__.index,columns=x__.columns)
x_train = pd.concat([x_train,firstb],axis=1)
x_val = pd.concat([x_val,firstbb],axis=1)


scaler1 = StandardScaler()
scaler1.fit(x___)
x_test = scaler1.transform(x___) 
x_test = pd.DataFrame(x_test,index=x___.index,columns=x___.columns)
x_test = pd.concat([x_test,firstbbb],axis=1)

In [12]:
x_val

,기온(C),강수량(mm),풍속(m/s),습도(%),연면적(m2),냉방면적(m2),month,day,hour,강수 유무,...,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
149856,0.668866,-0.138575,-0.044706,-0.294885,-0.004696,-0.039436,0.083570,-0.563913,0.0,-0.370045,...,0,0,0,0,0,0,1,0,0,0
163512,-1.022797,-0.138575,1.921331,-2.356188,-0.114148,-0.111546,-1.173679,-0.089758,0.0,-0.370045,...,0,0,0,0,0,0,0,1,0,0
168456,0.261614,-0.138575,-1.027724,0.563991,-0.047901,-0.079445,0.083570,0.621474,0.0,-0.370045,...,0,0,0,0,0,0,0,1,0,0
132888,-1.179433,-0.138575,-1.109642,0.134553,-0.122283,-0.171902,-1.173679,-0.208297,0.0,-0.370045,...,0,0,0,0,0,1,0,0,0,0
151368,-0.584218,-0.138575,-0.781969,0.478103,-0.135076,-0.124040,-1.173679,0.384397,0.0,-0.370045,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112272,-0.646872,-0.138575,0.282967,-0.810211,-0.171642,-0.165164,-1.173679,-1.275146,0.0,-0.370045,...,0,0,0,0,1,0,0,0,0,0
61704,0.230286,-0.138575,-0.618133,-1.067873,-0.045794,-0.060664,-1.173679,0.858552,0.0,-0.370045,...,1,0,0,0,0,0,0,0,0,0
108240,-2.275881,-0.138575,3.068186,1.165204,-0.118860,-0.110820,-1.173679,-1.038068,0.0,-0.370045,...,0,0,0,0,1,0,0,0,0,0
69168,0.167632,-0.138575,-0.536215,-0.123110,-0.196953,-0.163175,1.340820,0.265858,0.0,-0.370045,...,0,1,0,0,0,0,0,0,0,0


In [13]:
x_train = x_train.drop('hour',axis=1)
x_val = x_val.drop('hour',axis=1)
x_test = x_test.drop('hour',axis=1)

In [14]:
model = CatBoostRegressor(random_state=42)
model.fit(x_train,y_train)
y_pred = model.predict(x_val)
print(mean_absolute_percentage_error(y_val,y_pred))

Learning rate set to 0.055426
0:	learn: 1986.4863435	total: 173ms	remaining: 2m 53s
1:	learn: 1901.6547807	total: 178ms	remaining: 1m 29s
2:	learn: 1822.5338953	total: 184ms	remaining: 1m 1s
3:	learn: 1753.9187585	total: 190ms	remaining: 47.2s
4:	learn: 1683.8129578	total: 196ms	remaining: 38.9s
5:	learn: 1619.1235782	total: 201ms	remaining: 33.3s
6:	learn: 1564.3397783	total: 207ms	remaining: 29.4s
7:	learn: 1507.0216292	total: 212ms	remaining: 26.3s
8:	learn: 1459.7744132	total: 216ms	remaining: 23.7s
9:	learn: 1409.8092121	total: 219ms	remaining: 21.7s
10:	learn: 1366.0139725	total: 223ms	remaining: 20.1s
11:	learn: 1321.9737942	total: 227ms	remaining: 18.7s
12:	learn: 1284.7105961	total: 232ms	remaining: 17.6s
13:	learn: 1244.8785906	total: 237ms	remaining: 16.7s
14:	learn: 1209.9798296	total: 241ms	remaining: 15.8s
15:	learn: 1177.4309720	total: 247ms	remaining: 15.2s
16:	learn: 1142.4753155	total: 254ms	remaining: 14.7s
17:	learn: 1110.7527534	total: 260ms	remaining: 14.2s
18:	le

In [31]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train = gggg(data,building_info)
train1 = train.loc[train['hour']==23]
train1 = pd.get_dummies(train1,columns=['건물유형'],drop_first=True)

test1 = test123(test,building_info)
test1 = test1.loc[test1['hour']==23]
test1 = pd.get_dummies(test1,columns=['건물유형'],drop_first=True)


x=train1.drop('전력소비량(kWh)',axis=1)
y=train1[['전력소비량(kWh)']]

# x_train, x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=42)
        
firstb = x[["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
 '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트']]
x_ = x.drop(["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
             '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트'],axis=1)

# firstbb = x_val[["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
#  '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트']]
# x__ = x_val.drop(["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
#              '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트'],axis=1)



firstbbb = test1[["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
 '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트']]
x___ = test1.drop(["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
             '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트'],axis=1)

scaler = StandardScaler()
scaler.fit(x_)
x_train_ = scaler.transform(x_)
# x_val_ = scaler.transform(x__) 
x_train = pd.DataFrame(x_train_,index=x_.index,columns=x_.columns)
# x_val = pd.DataFrame(x_val_,index=x__.index,columns=x__.columns)
x_train = pd.concat([x_train,firstb],axis=1)
# x_val = pd.concat([x_val,firstbb],axis=1)


scaler1 = StandardScaler()
scaler1.fit(x___)
x_test = scaler1.transform(x___) 
x_test = pd.DataFrame(x_test,index=x___.index,columns=x___.columns)
x_test = pd.concat([x_test,firstbbb],axis=1)

x_train = x_train.drop('hour',axis=1)
x_test = x_test.drop('hour',axis=1)

model = CatBoostRegressor(random_state=42,verbose=0)
model.fit(x_train,y)
y_pred = model.predict(x_test)
data23 = y_pred

In [38]:
len(data0),len(data1)

(700, 700)

In [43]:
aa=[]
for i in range(700):
    aa.append(data0[i])
    aa.append(data1[i])
    aa.append(data2[i])
    aa.append(data3[i])
    aa.append(data4[i])
    aa.append(data5[i])
    aa.append(data6[i])
    aa.append(data7[i])
    aa.append(data8[i])
    aa.append(data9[i])
    aa.append(data10[i])
    aa.append(data11[i])
    aa.append(data12[i])
    aa.append(data13[i])
    aa.append(data14[i])
    aa.append(data15[i])
    aa.append(data16[i])
    aa.append(data17[i])
    aa.append(data18[i])
    aa.append(data19[i])
    aa.append(data20[i])
    aa.append(data21[i])
    aa.append(data22[i])
    aa.append(data23[i])
    

In [44]:
result(sample_submission,aa)

In [46]:
###############################################################################################################

In [232]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

data = data.loc[data['건물번호']==1]
test = test.loc[test['건물번호']==1]

train = gggg(data,building_info)
# train1 = pd.get_dummies(train1,columns=['건물유형'],drop_first=True)
train1 = train.drop('건물유형',axis=1)

test1 = test123(test,building_info)
# test1 = pd.get_dummies(test1,columns=['건물유형'],drop_first=True)
test1 = test1.drop('건물유형',axis=1)

x=train1.drop('전력소비량(kWh)',axis=1)
y=train1[['전력소비량(kWh)']]

# x_train, x_val,y_train,y_val = train_test_split(x,y,test_size=0.2,random_state=42)
        
firstb = x[["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','month','day','hour']]
x_ = x.drop(["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','month','day','hour'],axis=1)

# firstbb = x_val[["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
#  '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트']]
# x__ = x_val.drop(["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','건물유형_공공','건물유형_대학교','건물유형_데이터센터','건물유형_백화점및아울렛','건물유형_병원',
#              '건물유형_상용','건물유형_아파트','건물유형_연구소','건물유형_지식산업센터','건물유형_할인마트','건물유형_호텔및리조트'],axis=1)



firstbbb = test1[["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','month','day','hour']]
x___ = test1.drop(["불쾌지수 단계_1", '불쾌지수 단계_2', '불쾌지수 단계_3','month','day','hour'],axis=1)

scaler = StandardScaler()
scaler.fit(x_)
x_train_ = scaler.transform(x_)
# x_val_ = scaler.transform(x__) 
x_train = pd.DataFrame(x_train_,index=x_.index,columns=x_.columns)
# x_val = pd.DataFrame(x_val_,index=x__.index,columns=x__.columns)
x_train = pd.concat([x_train,firstb],axis=1)
# x_val = pd.concat([x_val,firstbb],axis=1)


scaler1 = StandardScaler()
scaler1.fit(x___)
x_test = scaler1.transform(x___) 
x_test = pd.DataFrame(x_test,index=x___.index,columns=x___.columns)
x_test = pd.concat([x_test,firstbbb],axis=1)

# x_train = x_train.drop('hour',axis=1)
# x_test = x_test.drop('hour',axis=1)

In [233]:
timestep= 24 #한번에 21주씩 고려하기 (batch?)

# x의 데이터 구조를 3차원으로 만들어줘야 합니다.
x_train = np.array([x_train[i : i + timestep] for i in range( len(x_train) - timestep ) ])
y = np.array([y.values[i + timestep,-1] for i in range( len(y) - timestep ) ])

# print(x)
# print(y)

x_train, x_val = x_train[:1608], x_train[1608:]
y_train, y_val = y[:1608], y[1608:]

# x_train = np.expand_dims(x_train, axis=1)
# x_val = np.expand_dims(x_val, axis=1)
# y_train = np.expand_dims(y_train, axis=1)
# y_val = np.expand_dims(y_val, axis=1)



print(x.shape, y.shape)
print('-------------------------------')
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
# print(x_test.shape, y_test.shape)
print('-------------------------------')
print('[ # , timestep, feature수 ] <-- 데이터의 구조 : ')

(2040, 15) (2016,)
-------------------------------
(1608, 24, 15) (1608,)
(408, 24, 15) (408,)
-------------------------------
[ # , timestep, feature수 ] <-- 데이터의 구조 : 


In [234]:
keras.backend.clear_session()

il = keras.layers.Input(shape=(24,15))
ll = keras.layers.LSTM(16, return_sequences=False)(il)
ll = keras.layers.BatchNormalization()(ll)
ol = keras.layers.Dense(1)(ll)

model = keras.models.Model(il,ol)
model.compile(loss=keras.losses.mean_absolute_percentage_error,metrics=['mse'],optimizer='adam')
model.summary()


es = EarlyStopping(monitor="val_loss",
                   min_delta=0.001,
                   verbose=1,
                   patience = 10,
                   restore_best_weights = True)
rl = ReduceLROnPlateau(monitor="val_loss",
                       patience=4,
                       factor=0.15,
                       verbose=1,
                       min_lr=0.0000001)

history = model.fit(x_train,y_train,callbacks=[es,rl],epochs=200,verbose=1,validation_data=[x_val,y_val])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24, 15)]          0         
                                                                 
 lstm (LSTM)                 (None, 16)                2048      
                                                                 
 batch_normalization (BatchN  (None, 16)               64        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 2,129
Trainable params: 2,097
Non-trainable params: 32
_________________________________________________________________
Epoch 1/200
51/51 [==============================] - 2s 11ms/step - loss: 99.9775 - mse: 8039958.5000 - val_loss: 99.9746 - 

In [235]:
x_test = np.array([x_test[i : i + timestep] for i in range( len(test) - timestep ) ])
x_test.shape

(144, 24, 15)

In [236]:
y_pred = model.predict(x_test)
y_pred


5/5 [==============================] - 0s 2ms/step


array([[ 4.80506897e+01],
       [ 9.22224884e+01],
       [ 1.09539665e+02],
       [ 1.12374535e+02],
       [ 1.04139168e+02],
       [ 7.88472214e+01],
       [ 4.65488243e+01],
       [ 1.96172714e+01],
       [ 1.80089390e+00],
       [-8.26423645e+00],
       [-1.13010082e+01],
       [-1.36293201e+01],
       [-1.44740505e+01],
       [-1.50810318e+01],
       [-1.51995926e+01],
       [-1.46903486e+01],
       [-1.45572548e+01],
       [-1.30756102e+01],
       [-9.42102146e+00],
       [-4.06037855e+00],
       [ 3.80364656e+00],
       [ 1.36948166e+01],
       [ 2.33226795e+01],
       [ 3.24862442e+01],
       [ 4.16195526e+01],
       [ 7.90607300e+01],
       [ 9.32145309e+01],
       [ 1.27528557e+02],
       [ 1.21709755e+02],
       [ 1.03588661e+02],
       [ 8.19411774e+01],
       [ 5.18910446e+01],
       [ 2.59838867e+01],
       [ 7.95343256e+00],
       [-1.99343061e+00],
       [-7.29343748e+00],
       [-9.54105186e+00],
       [-1.09134922e+01],
       [-1.0